In [9]:
# Import statements.
import sagemaker
from sagemaker.tensorflow.estimator import TensorFlow

In [10]:
# Global constants.
BUCKET_NAME = 'sagemaker-4bai-project'
S3_MODEL_DIR = 's3://{0}/model_3d'.format(BUCKET_NAME)

In [11]:
estimator = TensorFlow(py_version='py37',
                       framework_version='2.3',
                       model_dir=S3_MODEL_DIR,
                       role=sagemaker.get_execution_role(),
                       instance_count=1,
                       instance_type='ml.p2.xlarge',
                       entry_point='model_3d_train.py')

In [13]:
estimator = TensorFlow(py_version='py37',
                       framework_version='2.3',
                       model_dir=S3_MODEL_DIR,
                       role=sagemaker.get_execution_role(),
                       instance_count=1,
                       instance_type='ml.p2.xlarge',
                       image_uri='ec2-54-191-25-101.us-west-2.compute.amazonaws.com',
                       entry_point='model_3d_train.py')

In [12]:
estimator.fit()

ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.p2.xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please contact AWS support to request an increase for this limit.